
# Gibbs sampling

In this notebook, we show how to use CUQIpy to sample hierarchical Bayesian models using Gibbs sampling.

Gibbs sampling is a Markov chain Monte Carlo (MCMC) method for sampling a joint probability distribution of multiple random variables.
Instead of sampling all variables simultaneously, Gibbs sampling samples the variables sequentially. The sampling of each variable is achieved by sampling from the conditional distribution of that variable given (fixed, previously sampled) values of the other variables.

Gibbs sampling is often an efficient way of sampling from a joint distribution if the conditional distributions are easy to sample from. On the other hand, if the conditional distributions are highly correlated and/or are difficult to sample from, then Gibbs sampling can be very inefficient. For these reasons, Gibbs sampling is often a double-edged sword that needs to be used in the right context.

## Learning objectives

Going through this notebook you will see how to:

- Describe the basic idea of Gibbs sampling.
- Define a hierarchical Bayesian model using CUQIpy.
- Define a Gibbs sampling scheme in CUQIpy.
- Run the Gibbs sampler and analyze the results.

## Table of contents

1. [The basics of Gibbs sampling](#basics)
2. [Gibbs for inverse problems](#inverse)
3. [Exploring other priors](#exploring-other-priors)
4. [Connection to BayesianProblem class](#BayesianProblem)


<div style="border: 2px solid #FFB74D; background-color: #FFF3E0; border-radius: 8px; padding: 10px; font-family: Arial, sans-serif; color: #333; box-shadow: 2px 2px 8px rgba(0, 0, 0, 0.1); max-width: 750px; margin: 0 auto;">
  <strong style="color: #E65100;">⚠️ Note:</strong> This notebook uses MCMC samplers from the new <code>cuqi.experimental.mcmc</code> module, which are expected to become the default soon. Check out the <a href="https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.experimental.mcmc.html#module-cuqi.experimental.mcmc">documentation</a> for more details.
</div>

## Setup
We start by importing the necessary modules. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import cuqi
from cuqi.testproblem import Deconvolution1D
from cuqi.distribution import Gaussian, Gamma, JointDistribution, GMRF, LMRF
from cuqi.experimental.mcmc import HybridGibbs, LinearRTO, Conjugate, UGLA, ConjugateApprox, Direct
from cuqi.problem import BayesianProblem

# 1. The basics of Gibbs sampling <a class="anchor" id="basics"></a>

To begin, let us consider a very simple Bayesian problem with two random variables, $d$ and $x$.

$$
\begin{align*}
d &\sim \mathrm{Gamma}(1, 1)\\
x &\sim \mathrm{Gaussian}(0, d^{-1}).
\end{align*}
$$

As we have already seen in the previous notebooks, we can define the distributions in CUQIpy as follows:


In [ ]:
d = Gamma(1, 1)
x = Gaussian(0, cov=lambda d: 1/d)

Our goal is to sample from the joint distribution $p(d, x)$. We can think of this as our target distribution. 

**Note** In this contrived example, we can actually sample from the joint distribution directly. However, we will use this example to illustrate the basic idea of Gibbs sampling.

We define the joint distribution in CUQIpy as follows:

In [ ]:
target = JointDistribution(x, d)

print(target)

One idea for sampling $p(d,x)$ is to define the parameter $\boldsymbol{\theta} = [d; x]$ and sample from distribution $p(\boldsymbol{\theta})$ at the same time. However, this is only possible for simple models, where the joint distribution is easy to sample from.

Gibbs sampling is an alternative approach, where we iteratively sample each variable from its conditional distribution, given the values of all other variables in the distribution. For this task with $p(d,x)$, we would sample from $p(d|x)$ and $p(x|d)$ iteratively. We can summarize this procedure as repeating the following steps:

$$
\begin{align*}
\text{1. draw } d^{(i+1)} &\sim p(d \mid x^{(i)})\\
\text{2. draw } x^{(i+1)} &\sim p(x \mid d^{(i+1)})
\end{align*}
$$

where $x^{(i)}$ and $d^{(i)}$ are the values of $x$ and $d$ at iteration $i$. It can be shown that samples drawn from the above scheme will be distributed according to $p(d, x)$.

## 1.1 Defining a sampling strategy

When the conditional distributions are easy to sample from, Gibbs sampling can be very efficient. In the example above, we aim to sample from the following conditional distributions:

1. Sample $x^{(i+1)}$ from $p(x \mid d^{(i)})$,
2. Sample $d^{(i+1)}$ from $p(d \mid x^{(i+1)})\propto L(d\mid x^{(i+1)})p(d)$,

where $L(d\mid x):=p(x\mid d)$ is the likelihood function with respect to $d$.

**For step 1:** Note that we already know that $p(x\mid d)=\mathrm{Gaussian}(0, d^{-1})$. Hence, we can directly sample from $x \mid  d$ by sampling from $\mathrm{Gaussian}(0, d^{-1})$.

**For step 2:** For $p(d\mid x)$ we could use any MCMC sampler to sample with. However, note that the Gaussian and Gamma distributions are conjugate, so we can recognize $p(d\mid x)$ as a Gamma distribution:

$$
\begin{align*}
p(d\mid x) &\propto L(d\mid x)p(d)\\
&= \exp\left(-\frac{d}{2}x^2\right)\frac{\beta^\alpha}{\Gamma(\alpha)}d^{\alpha-1}\exp(-\beta d)\\
&= \mathrm{Gamma}(\alpha+1/2, \beta+\frac{1}{2}x^2)
\end{align*}
$$

where $\alpha=1$ and $\beta=1$ are the original parameters of the Gamma distribution for $d$.

Instead of manually deriving the Gamma distribution for the conjugate posterior, CUQIpy provides a `Conjugate` sampler, which can be used to automatically sample from any (supported) conjugate distributions. Similarly we can use the `Direct` sampler to sample from the Gaussian distribution.

*The interface and design of the `Conjugate` and `Direct` samplers is still under development and may change in the future.*

In summary, we define a Gibbs sampling scheme in CUQIpy as follows:

In [ ]:
# Define the sampling strategy
sampling_strategy = {
    'x' : Direct(),
    'd' : Conjugate(),
}

We can then define the Gibbs sampler and sample from the joint distribution as follows:

In [ ]:
# Gibbs sampler
sampler = HybridGibbs(target, sampling_strategy)

# Sample
sampler.warmup(1000)
sampler.sample(1000)

# Get samples
samples = sampler.get_samples()


And we can visualize e.g. the trace plots as follows:

In [ ]:
samples["x"].plot_trace()
samples["d"].plot_trace()

# 2. Gibbs for inverse problems <a class="anchor" id="inverse"></a>

The above example was a simple example to illustrate the basic idea of Gibbs sampling. However, often for these simple examples there exist more efficient sampling methods. In the following we instead focus on inverse problems where Gibbs sampling will prove to be useful, because efficient samplers are not readily available."

## 2.1 Deterministic forward model and data
Consider a Bayesian inverse problem

$$
\mathbf{y}=\mathbf{A}\mathbf{x},
$$

where $\mathbf{A}: \mathbb{R}^n \to \mathbb{R}^m$ is the forward model of the inverse problem and $\mathbf{y}$ and $\mathbf{x}$ are random variables for the data and unknown parameter, respectively.

In this case, we assume that the forward model is a convolution operator. We can load this example from the testproblem library of CUQIpy and visualize the true solution (sharp signal) and data (convolved signal).

In [ ]:
# Model and data
A, y_data, probinfo = Deconvolution1D(phantom='sinc', noise_std=0.005, PSF_param=6).get_components()

# Get dimension of signal
n = A.domain_dim

# Plot exact solution and observed data
plt.subplot(121)
probinfo.exactSolution.plot()
plt.title('Exact solution')

plt.subplot(122)
y_data.plot()
plt.title("Observed data")

# Print problem information
print(probinfo)

## 2.2 Hierarchical Bayesian model

We are now going to define a hierarchical Bayesian model for the inverse problem.

First, we are going to model the prior distribution of the parameter $\mathbf{x}$ as a Gaussian Markov random field (GMRF), i.e.

$$
\begin{align*}
\mathbf{x} \mid d &\sim \mathrm{GMRF}(\mathbf{0}, d).
\end{align*}
$$

This is a built-in distribution in CUQIpy, which models the difference between neighboring elements as a Gaussian, see [GMRF](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.distribution/cuqi.distribution.GMRF.html#cuqi.distribution.GMRF) for more details.

We do not know a good choice of precision parameter $d$ for this prior, so we make use of `lambda` (anonymous) functions to make $d$ a conditioning variable of the distribution. The distribution is defined in CUQIpy as follows.


In [ ]:
# Define prior
x = GMRF(np.zeros(n), lambda d: d)

print(x)

#### ★ Try yourself (optional):  

Try computing some samples from the GMRF distribution and visualize them. You can use the `sample` method of the `GMRF` distribution. You can also visualize the samples using the `plot` method of the obtained samples.

Hint: Since $d$ is a conditioning variable, you need to specify the value of $d$ when sampling.

In [ ]:
# Your code here



The observed data is corrupted by additive Gaussian noise, so for random variable $\mathbf{y}$ we have

$$\mathbf{y} \mid \mathbf{x}, s \sim \mathrm{Gaussian}(\mathbf{A} \mathbf{x}, s^{-1}\mathbf{I}_m),$$

where for this example, we pretend also not to know the noise precision $s$.

Similar to the prior distribution, we can define this distribution as follows.

In [ ]:
# Data distribution (likelihood)
y = Gaussian(A@x, cov=lambda s: 1/s)

print(y)

To model the two *hyperparameters* $d$ and $s$, we can use a weakly informative `Gamma` distribution, i.e. a distribution with a wide range of possible values.

$$
\begin{align*}
        d &\sim \mathrm{Gamma}(1, 10^{-4}) \\
        s &\sim \mathrm{Gamma}(1, 10^{-4})
\end{align*}
$$


In [ ]:
d = Gamma(1, 1e-4)
s = Gamma(1, 1e-4)

print(d)
print(s)

To see that the Gamma hyperprior is weakly informative, we can visualize the pdf of the Gamma distribution. Notice that the pdf is mostly flat for the range of values from $0$ to $10^4$. This means that the Gamma distribution is fairly uninformative for this range of values and assign equal probability to all values in this range.


In [ ]:
grid = np.linspace(1e-4, 1e+4, 1000)
plt.plot(grid, [d.pdf(val) for val in grid])
plt.ylim(0, 2*1e-4)
plt.title("PDF of Gamma(1, 1e-4)");

In total, we can summarize the hierarchical Bayesian problem as follows (without explicitly writing the conditioning variables):

$$
\begin{align*}
        d &\sim \mathrm{Gamma}(1, 10^{-4}) \\
        s &\sim \mathrm{Gamma}(1, 10^{-4}) \\
        \mathbf{x} &\sim \mathrm{GMRF}(\mathbf{0}, d) \\
        \mathbf{y} &\sim \mathrm{Gaussian}(\mathbf{A} \mathbf{x}, s^{-1} \mathbf{I}_m)
\end{align*}
$$

which in CUQIpy matches line-by-line:

In [ ]:
d = Gamma(1, 1e-4)
s = Gamma(1, 1e-4)
x = GMRF(np.zeros(n), lambda d: d)
y = Gaussian(A@x, cov=lambda s: 1/s)

## Posterior distribution

We now define the posterior distribution by first creating the joint distribution $p(\mathbf{y}, \mathbf{x}, d, s)$ and then conditioning on the observed data $\mathbf{y}_\mathrm{data}$ to obtain the posterior which can be derived in density form as

$$
\begin{align*}
p(\mathbf{x}, d, s \mid \mathbf{y}_\mathrm{data}) = L(\mathbf{x}, s \mid \mathbf{y}_\mathrm{data}) p(\mathbf{x} \mid d) p(d) p(s).
\end{align*}
$$

This in done in CUQIpy as follows

In [ ]:
# Create joint distribution
joint = JointDistribution(y, x, d, s)

# Define posterior by conditioning on the data
posterior = joint(y=y_data) # Automatically applies Bayes rule

# View the posterior
print(posterior)

Notice that after conditioning on the data, the distribution associated with $\mathbf{y}$ became a likelihood function and that the posterior is now a joint distribution of the variables $\mathbf{x}$, $d$ and $s$. It matches the mathematical expression above.



#### ★ Try yourself (optional):  
1. Try evaluating the un-normalized log probability density (logd) of the posterior distribution $p(\mathbf{x}, d, s \mid \mathbf{y}_\mathrm{data})$ at some points $\mathbf{x}^*\in \mathbb{R}^n$, $d^*\in \mathbb{R}_+$ and $s^*\in \mathbb{R}_+$. Hint: The `logd` method of the `Joint` distribution takes comma-separated keyword arguments for the conditioning variables. For example, `logd(x=xstar, d=dstar, s=sstar)`.

2. You can also try *conditioning* the posterior distribution on some of the variables, e.g. `posterior(x=xstar, d=dstar)`. This should compute the marginal distribution of $s$ given $\mathbf{x}^*$ and $d^*$, i.e. $p(s \mid \mathbf{y}_\mathrm{data}, \mathbf{x}^*, d^*)$.

3. Experiment with sampling from a similar BIP that does not include hyperparameters $d$ and $s$ (use the values $d=1$ and $s=10000$ to determine the precision of the prior and the likelihood). You can create the BIP as follows. Hint: use the `UQ` method of the `BayesianProblem` to sample from the posterior distribution.

```python
x2 = GMRF(np.zeros(n), prec=1)
y2 = Gaussian(A@x2, prec=10000)
BP2 = BayesianProblem(x2, y2)
BP2.set_data(y2=y_data)
```

In [ ]:
# Your code here



## 2.3 Gibbs Sampler

The hierarchical Bayesian problem above has some important properties that we
can exploit to make the sampling more efficient.

First, note that for the pair of a Gaussian distribution with a Gamma distribution
for the precision is a conjugate pair (as we also saw earlier). This means that the
posterior distribution with respect to the parameter for the precision is also a
Gamma distribution. This means that we can efficiently sample from $d$ and $s$
conditional on the other variables by exploiting conjugacy.

Second, note that the prior distribution of $\mathbf{x}$ is
a Gaussian Markov random field (GMRF) and that the distribution for
$\mathbf{y}$ is also Gaussian with a Linear operator acting
on $\mathbf{x}$ as the mean variable. This means that we can
efficiently sample from $\mathbf{x}$ conditional on the other
variables using the ``LinearRTO`` sampler.

Taking these two facts into account, we can define a Gibbs sampler
that uses the ``Conjugate`` sampler for $d$ and $s$ and
the ``LinearRTO`` sampler for $\mathbf{x}$.

This is done in CUQIpy as follows:



In [ ]:
# Define sampling strategy
sampling_strategy = {
    'x': LinearRTO(),
    'd': Conjugate(),
    's': Conjugate()
}

# Define Gibbs sampler
sampler = HybridGibbs(posterior, sampling_strategy)

# Run sampler
sampler.warmup(200)
sampler.sample(1000)

# Get samples
samples = sampler.get_samples() # ToDo. Allow removal of burn-in in get_samples method

## 2.4 Analyze results

After sampling we can inspect the results. The samples are stored
as a dictionary with the variable names as keys. Samples for each 
variable is stored as a CUQIpy Samples object which contains the
many convenience methods for diagnostics and plotting of MCMC samples.



In [ ]:
# Plot credible intervals for x
samples['x'].plot_ci(exact=probinfo.exactSolution)

Trace plot for d



In [ ]:
samples['d'].plot_trace(figsize=(8,2))

Trace plot for s



In [ ]:
samples['s'].plot_trace(figsize=(8,2), exact=1/0.005**2)

Notice that the true noise standard deviation was $0.005$ which translates to a precision of $s=40000$. The trace plot for $s$ shows that the sampler has converged close to the true value.

# 3 Exploring other priors <a class="anchor" id="exploring-other-priors"></a>

Suppose the true solution was not modelled well by a GMRF prior. This would for example be the case for a piece-wise constant "square" signal. We can quickly try that experiment by generating new data from the test problem with a `square` phantom.

For illustration we repeat all the code steps from above in a single cell.


In [ ]:
# Deterministic forward model + data
A, y_data, probinfo = Deconvolution1D(phantom='square').get_components()

# Bayesian model
d = Gamma(1, 1e-4)
s = Gamma(1, 1e-4)
x = GMRF(np.zeros(n), lambda d: d)
y = Gaussian(A@x, cov=lambda s: 1/s)

# Joint distribution
joint = JointDistribution(y, x, d, s)

# Posterior
posterior = joint(y=y_data)

# Sampling strategy
sampling_strategy = {
    'x': LinearRTO(),
    'd': Conjugate(),
    's': Conjugate()
}

# Gibbs sampler
sampler = HybridGibbs(posterior, sampling_strategy)

# Run sampler
sampler.warmup(200)
sampler.sample(1000)

# Get samples
samples = sampler.get_samples() # ToDo. Allow removal of burn-in in get_samples method

# Plot credible intervals for x
samples['x'].plot_ci(exact=probinfo.exactSolution)

Notice that while the sampling still ran, the posterior distribution does not match the characteristics of the square phantom. We can improve this result by switching to a prior that better matches this kind of signal.

One choice is the [LMRF](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.distribution/cuqi.distribution.LMRF.html) prior, which assumes a Laplace distribution for the differences between neighboring elements of $\mathbf{x}$ instead of a Gaussian. That is we assume

$$
\begin{align*}
\mathbf{x} \sim \text{LMRF}(\mathbf{0}, d^{-1}),
\end{align*}
$$

which translates to the assumption that $x_i-x_{i-1} \sim \mathrm{Laplace}(0, d^{-1})$ for $i=1, \ldots, n$ (excluding boundaries)

This prior is implemented in CUQIpy as the ``LMRF`` distribution.
To update our model we simply need to replace the ``GMRF`` distribution
with the ``LMRF`` distribution. Note that the Laplace distribution
is defined via a scale parameter, so we invert the parameter $d$.

This laplace distribution and new posterior can be defined as follows:



In [ ]:
# Define new distribution for x
x = LMRF(np.zeros(n), lambda d: 1/d)

# Define new joint distribution with piecewise constant prior
joint_Ld = JointDistribution(d, s, x, y)

# Define new posterior by conditioning on the data
posterior_Ld = joint_Ld(y=y_data)

print(posterior_Ld)

## 3.1 Gibbs Sampler (with Laplace prior)

Using the same approach as ealier we can define a Gibbs sampler
for this new hierarchical model. The only difference is that we
now need to use a different sampler for $\mathbf{x}$ because
the ``LinearRTO`` sampler only works for Gaussian distributions.

In this case we use the [UGLA](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.sampler/cuqi.sampler.UGLA.html) sampler
for $\mathbf{x}$. We also use an approximate Conjugate
sampler for $d$ which approximately samples from the
posterior distribution of $d$ conditional on the other
variables in an efficient manner. For more details see e.g.
[Uribe et al. (2021)](https://arxiv.org/abs/2104.06919).



In [ ]:
# Define sampling strategy
sampling_strategy = {
    'x': UGLA(),
    'd': ConjugateApprox(),
    's': Conjugate()
}

# Define Gibbs sampler
sampler_Ld = HybridGibbs(posterior_Ld, sampling_strategy)

# Run sampler
sampler_Ld.warmup(200)
sampler_Ld.sample(1000)

# Get samples
samples_Ld = sampler_Ld.get_samples() 
samples_Ld = {key: samples_Ld[key].burnthin(200) for key in samples_Ld} # ToDo. Allow removal of burn-in in get_samples method

## 3.2 Analyze results

Again we can inspect the results.
Here we notice the posterior distribution matches the exact solution better than before.



In [ ]:
# Plot credible intervals for the signal
samples_Ld['x'].plot_ci(exact=probinfo.exactSolution) # Issue here?

In [ ]:
samples_Ld['d'].plot_trace(figsize=(8,2))

In [ ]:
samples_Ld['s'].plot_trace(figsize=(8,2), exact=1/0.01**2)

We can also view some posterior samples.

In [ ]:
samples_Ld['x'].plot()

# 4. Connection to BayesianProblem class <a class="anchor" id="BayesianProblem"></a>

Finally, let us connect back the non-expert interface through the `BayesianProblem` class.

Instead of explicitly defining the Gibbs sampler, we can also (for supported cases) simply provide the distributions for each of the parameters and the observed data to the `BayesianProblem` class. The class will then automatically construct a Gibbs sampler for the problem. This is done as follows:

In [ ]:
# Provide distributions for parameters and set data
BP = BayesianProblem(y,x,d,s).set_data(y=y_data)

# DO UQ and compare with exact solution and noise precision
samples_BP = BP.UQ(exact={"x":probinfo.exactSolution, "s":1/0.01**2}, experimental=True)

#### ★ Try yourself (optional):  

Try building your own Bayesian problem and sampling from it using the `BayesianProblem` class. You can use the code from the previous sections as a starting point. If the automatic sampler selection fails, you can also try to defining the Gibbs sampler manually.

You can also try the same Bayesian problem but for [2D deconvolution](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.testproblem/cuqi.testproblem.Deconvolution2D.html#cuqi.testproblem.Deconvolution2D)

In [ ]:
# Your code here


##### <font color=#8B4513> ★ Exercise </font>

Numerical verification of the conjugacy of a Gaussian likelihood a Gaussian prior. Consider the following simple BIP:
```python
# Prior
cov1 = np.array([[1, 0.5], [0.5, 1]])
x1 = Gaussian(np.zeros(2), cov1)
I = np.eye(2)
cov_obs = 0.1*I

# Forward model and likelihood
A = LinearModel(forward=I)
x2 = Gaussian(A@x1, cov_obs)

# Set the BIP
BP2 = BayesianProblem(x1, x2)
data = np.array([0.1, 0.2])
BP2.set_data(x2=data)

# Solve the BIP
samples_post = BP2.sample_posterior(10000, experimental=True)
samples_post.plot_pair()
```

Create a `CUQIpy` Gaussian distribution `x3` that is equivalent to the posterior distribution of this BIP. Hint: recall conjugacy of Gaussian likelihood and Gaussian prior. Sample `x3` and use the `plot_pair` method to visualize the samples. Compare the results with the samples from the `BayesianProblem` class.

In [ ]:
# Your code here